In [1]:
# 7c – Generalized Linear Model (GLM) Forecast Approach
# -----------------------------------------------------

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


In [2]:
# Load the data

# Paths
processed_path = (
    "/Users/rosstaylor/Downloads/Research Project/Code Folder/"
    "nhs-diagnostics-dids-eda/nhs-dids-explorer/data/processed/"
    "demand_distributions/modality_demand_by_age_and_source.csv"
)
raw_path = (
    "/Users/rosstaylor/Downloads/Research Project/Code Folder/"
    "nhs-diagnostics-dids-eda/nhs-dids-explorer/data/raw/all_icbs_2024.csv"
)

# DataFrames
df_demand = pd.read_csv(processed_path)   # processed demand distribution
df_icb    = pd.read_csv(raw_path)         # raw ICB-level data


In [3]:
# Cell 2 – Inspect each DataFrame
for name, df in [("df_demand", df_demand), ("df_icb", df_icb)]:
    print(f"\n{name}")
    print("-" * len(name))
    print(f"Shape   : {df.shape}")        # (rows, columns)
    print(f"Columns : {list(df.columns)}")
    display(df.head())                    # first 5 rows (Jupyter/IPython)
    # If you prefer a concise summary, uncomment the next line:
    # df.info(show_counts=True)



df_demand
---------
Shape   : (1293, 4)
Columns : ['age', 'modality', 'referral_type', 'procedure_count']


,age,modality,referral_type,procedure_count
0,0.0,CT,Emergency,212
1,0.0,CT,GP,2
2,0.0,CT,Inpatient,443
3,0.0,CT,Other/Unknown,25
4,0.0,CT,Outpatient,103



df_icb
------
Shape   : (3475, 22)
Columns : ['lsoa21cd', 'lsoa21nm', 'ICB23NM', 'total_population', 'age_0_4', 'age_5_9', 'age_10_14', 'age_15_19', 'age_20_24', 'age_25_29', 'age_30_34', 'age_35_39', 'age_40_44', 'age_45_49', 'age_50_54', 'age_55_59', 'age_60_64', 'age_65_69', 'age_70_74', 'age_75_79', 'age_80_84', 'age_85_plus']


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age_0_4,age_5_9,age_10_14,age_15_19,age_20_24,age_25_29,...,age_40_44,age_45_49,age_50_54,age_55_59,age_60_64,age_65_69,age_70_74,age_75_79,age_80_84,age_85_plus
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,43.76,70.21,75.30,87.51,63.09,45.79,...,64.11,85.48,108.88,113.97,126.18,105.83,113.97,80.39,63.09,51.90
1,E01020481,Dorset 042D,NHS Dorset Integrated Care Board,1347.30,39.69,32.56,40.70,31.55,52.92,47.83,...,51.90,58.00,69.20,118.04,160.78,133.31,161.80,105.83,61.06,83.44
2,E01020482,Dorset 042E,NHS Dorset Integrated Care Board,1585.42,70.21,77.34,74.28,47.83,67.16,77.34,...,89.55,89.55,92.60,129.24,150.60,103.80,132.29,94.64,49.86,52.92
3,E01020479,Dorset 042B,NHS Dorset Integrated Care Board,1150.91,33.58,32.56,32.56,46.81,33.58,36.63,...,29.51,50.88,50.88,77.34,78.36,105.83,106.85,120.08,94.64,155.69
4,E01020478,Dorset 042A,NHS Dorset Integrated Care Board,1129.54,37.65,44.77,51.90,59.02,47.83,32.56,...,56.99,55.97,82.43,104.81,92.60,89.55,94.64,83.44,48.84,55.97


In [4]:
# Identify age band columns
age_band_cols = [col for col in df_icb.columns if col.startswith("age_")]

# Melt to long format: each row becomes LSOA × age band
df_long = df_icb.melt(
    id_vars=["lsoa21cd", "lsoa21nm", "ICB23NM", "total_population"],
    value_vars=age_band_cols,
    var_name="age_band",
    value_name="band_population"
)


In [5]:
# Define mapping: expand each band into list of individual ages
age_band_to_ages = {
    "age_0_4":    list(range(0, 5)),
    "age_5_9":    list(range(5, 10)),
    "age_10_14":  list(range(10, 15)),
    "age_15_19":  list(range(15, 20)),
    "age_20_24":  list(range(20, 25)),
    "age_25_29":  list(range(25, 30)),
    "age_30_34":  list(range(30, 35)),
    "age_35_39":  list(range(35, 40)),
    "age_40_44":  list(range(40, 45)),
    "age_45_49":  list(range(45, 50)),
    "age_50_54":  list(range(50, 55)),
    "age_55_59":  list(range(55, 60)),
    "age_60_64":  list(range(60, 65)),
    "age_65_69":  list(range(65, 70)),
    "age_70_74":  list(range(70, 75)),
    "age_75_79":  list(range(75, 80)),
    "age_80_84":  list(range(80, 85)),
    "age_85_plus": list(range(85, 111))  # Assume upper limit of 110
}

# Expand rows: divide band population equally across ages
expanded_rows = []

for _, row in df_long.iterrows():
    ages = age_band_to_ages.get(row["age_band"], [])
    if not ages:
        continue
    per_age_pop = row["band_population"] / len(ages)
    for age in ages:
        expanded_rows.append({
            "lsoa21cd": row["lsoa21cd"],
            "lsoa21nm": row["lsoa21nm"],
            "ICB23NM": row["ICB23NM"],
            "total_population": row["total_population"],
            "age": age,
            "population": per_age_pop
        })

df_age_expanded = pd.DataFrame(expanded_rows)


In [6]:
# Basic shape and columns
print(f"Expanded DataFrame shape: {df_age_expanded.shape}")
print(f"Expected columns: {list(df_age_expanded.columns)}")

# Check number of age rows per LSOA
rows_per_lsoa = df_age_expanded.groupby("lsoa21cd")["age"].count()
print("\nRows per LSOA (should be around 111 each):")
display(rows_per_lsoa.describe())

# Check population sums
total_original = df_icb["total_population"].sum()
total_expanded = df_age_expanded.groupby("lsoa21cd")["population"].sum().sum()
print(f"\nOriginal total population: {total_original:,.2f}")
print(f"Expanded population sum  : {total_expanded:,.2f}")
print(f"Difference (should be small): {total_original - total_expanded:,.2f}")

# Spot check one LSOA
sample = df_age_expanded[df_age_expanded["lsoa21cd"] == df_age_expanded["lsoa21cd"].iloc[0]]
print("\nSample LSOA (first 5 rows):")
display(sample.head())


Expanded DataFrame shape: (385725, 6)
Expected columns: ['lsoa21cd', 'lsoa21nm', 'ICB23NM', 'total_population', 'age', 'population']

Rows per LSOA (should be around 111 each):


count    3472.000000
mean      111.095910
std         3.261885
min       111.000000
25%       111.000000
50%       111.000000
75%       111.000000
max       222.000000
Name: age, dtype: float64


Original total population: 5,913,246.55
Expanded population sum  : 5,913,246.30
Difference (should be small): 0.25

Sample LSOA (first 5 rows):


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age,population
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,0,8.752
1,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,1,8.752
2,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,2,8.752
3,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,3,8.752
4,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,4,8.752


In [7]:
# Ensure 'age' in df_demand is integer
df_demand["age"] = df_demand["age"].astype(int)

# Inspect unique combinations
print("Unique combinations in df_demand:")
print(df_demand[["age", "modality", "referral_type"]].drop_duplicates().shape)


Unique combinations in df_demand:
(1293, 3)


In [8]:
# Merge on 'age' to get all possible combinations for each LSOA-age pair
df_merged = df_age_expanded.merge(
    df_demand,
    on="age",
    how="inner"  # Only keep ages present in both
)


In [9]:
# Shape and column check
print(f"Merged DataFrame shape: {df_merged.shape}")
print(f"Expected columns: {list(df_merged.columns)}")

# Check number of modalities and referral types per LSOA-age
check = df_merged.groupby(["lsoa21cd", "age"]).agg(
    modalities=("modality", "nunique"),
    referral_types=("referral_type", "nunique")
)
print("\nModalities and referral types per LSOA-age (first 5):")
display(check.head())

# Spot check for one LSOA
sample = df_merged[df_merged["lsoa21cd"] == df_merged["lsoa21cd"].iloc[0]]
print("\nSample merged rows:")
display(sample.head())

# Confirm population and demand alignment
print("\nSample row population × procedure_count (proxy for expected procedures):")
sample["expected_procedures"] = sample["population"] * sample["procedure_count"] / 1000
display(sample[["age", "modality", "referral_type", "population", "procedure_count", "expected_procedures"]].head())


Merged DataFrame shape: (4493175, 9)
Expected columns: ['lsoa21cd', 'lsoa21nm', 'ICB23NM', 'total_population', 'age', 'population', 'modality', 'referral_type', 'procedure_count']

Modalities and referral types per LSOA-age (first 5):


modalities  referral_types
lsoa21cd  age                            
E01014014 0             3               5
          1             3               4
          2             3               5
          3             3               5
          4             2               5


Sample merged rows:


,lsoa21cd,lsoa21nm,ICB23NM,total_population,age,population,modality,referral_type,procedure_count
0,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,0,8.752,CT,Emergency,212
1,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,0,8.752,CT,GP,2
2,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,0,8.752,CT,Inpatient,443
3,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,0,8.752,CT,Other/Unknown,25
4,E01020484,Dorset 042G,NHS Dorset Integrated Care Board,1444.99,0,8.752,CT,Outpatient,103



Sample row population × procedure_count (proxy for expected procedures):


,age,modality,referral_type,population,procedure_count,expected_procedures
0,0,CT,Emergency,8.752,212,1.855424
1,0,CT,GP,8.752,2,0.017504
2,0,CT,Inpatient,8.752,443,3.877136
3,0,CT,Other/Unknown,8.752,25,0.218800
4,0,CT,Outpatient,8.752,103,0.901456


Step A Summary: Simple Poisson GLM

In [11]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Work on a copy to preserve raw data
df_glm = df_merged.copy()

# Filter out rows with zero expected_procedures (optional, for stability)
df_glm["expected_procedures"] = (
    df_glm["population"] * df_glm["procedure_count"] / 1000
)
df_glm = df_glm[df_glm["expected_procedures"] > 0].copy()


In [12]:
# Keep only essential columns
df_glm_trimmed = df_glm[["age", "modality", "referral_type", "expected_procedures"]].copy()

# Sample 5% of rows to speed up model fitting
df_glm_sample = df_glm_trimmed.sample(frac=0.05, random_state=42)

# Define and fit Poisson GLM
formula = "expected_procedures ~ age + C(modality) + C(referral_type)"
poisson_model = smf.glm(
    formula=formula,
    data=df_glm_sample,
    family=sm.families.Poisson()
).fit()

# Show model summary
print(poisson_model.summary())

                  Generalized Linear Model Regression Results                  
Dep. Variable:     expected_procedures   No. Observations:               224477
Model:                             GLM   Df Residuals:                   224469
Model Family:                  Poisson   Df Model:                            7
Link Function:                     Log   Scale:                          1.0000
Method:                           IRLS   Log-Likelihood:            -2.2142e+06
Date:                 Wed, 02 Jul 2025   Deviance:                   3.8028e+06
Time:                         09:33:15   Pearson chi2:                 4.32e+06
No. Iterations:                      8   Pseudo R-squ. (CS):              1.000
Covariance Type:             nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [13]:
# Calculate dispersion statistic: Pearson chi2 / degrees of freedom
pearson_chi2 = ((poisson_model.resid_pearson) ** 2).sum()
dispersion = pearson_chi2 / poisson_model.df_resid
print(f"Dispersion: {dispersion:.2f}")


Dispersion: 19.24


Step B – Aggregation and Negative Binomial Model

In [32]:
# Define formula and fit
formula_b = "expected_procedures ~ C(modality) + C(referral_type)"

nb_model = smf.glm(
    formula=formula_b,
    data=df_b,
    family=sm.families.NegativeBinomial()
).fit()

# Show summary
print(nb_model.summary())

                  Generalized Linear Model Regression Results                  
Dep. Variable:     expected_procedures   No. Observations:                52050
Model:                             GLM   Df Residuals:                    52043
Model Family:         NegativeBinomial   Df Model:                            6
Link Function:                     Log   Scale:                          1.0000
Method:                           IRLS   Log-Likelihood:            -3.5579e+05
Date:                 Wed, 02 Jul 2025   Deviance:                       71680.
Time:                         09:53:30   Pearson chi2:                 4.73e+04
No. Iterations:                     52   Pseudo R-squ. (CS):             0.9125
Covariance Type:             nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

Step C: Final Age-Level Model with Negative Binomial + Population Offset

In [34]:
# Clean dataframe for modelling
df_model = df_glm[[
    "age", "modality", "referral_type", "population", "expected_procedures"
]].copy()

# Drop any rows where population is zero or NaN (avoid log(0))
df_model = df_model[df_model["population"] > 0].copy()

# Add offset term: log of population
df_model["log_population"] = np.log(df_model["population"])


In [35]:
# Fit model: expected count ~ age + modality + referral_type, offset = log(pop)
nb_model = smf.glm(
    formula="expected_procedures ~ age + C(modality) + C(referral_type)",
    data=df_model,
    family=sm.families.NegativeBinomial(),
    offset=df_model["log_population"]
).fit()

# View summary
print(nb_model.summary())


                  Generalized Linear Model Regression Results                  
Dep. Variable:     expected_procedures   No. Observations:              4489534
Model:                             GLM   Df Residuals:                  4489526
Model Family:         NegativeBinomial   Df Model:                            7
Link Function:                     Log   Scale:                          1.0000
Method:                           IRLS   Log-Likelihood:            -1.1569e+07
Date:                 Wed, 02 Jul 2025   Deviance:                   3.3752e+06
Time:                         09:57:04   Pearson chi2:                 3.31e+06
No. Iterations:                     19   Pseudo R-squ. (CS):             0.7629
Covariance Type:             nonrobust                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [36]:
# Predict directly from the model using the offset
df_model["volume_predicted"] = nb_model.predict(df_model, offset=df_model["log_population"])


In [37]:
# Calculate error ratio
df_model["error_ratio"] = df_model["volume_predicted"] / df_model["expected_procedures"]

# Summarise prediction accuracy
print(df_model["error_ratio"].describe())

# Optional: view example rows
display(df_model[[
    "age", "modality", "referral_type", "population",
    "expected_procedures", "volume_predicted", "error_ratio"
]].head())


count    4.489534e+06
mean     4.515384e+01
std      2.916889e+02
min      7.779268e-02
25%      6.262242e-01
50%      1.478769e+00
75%      4.591591e+00
max      4.241917e+03
Name: error_ratio, dtype: float64


,age,modality,referral_type,population,expected_procedures,volume_predicted,error_ratio
0,0,CT,Emergency,8.752,1.855424,2.598663,1.400576
1,0,CT,GP,8.752,0.017504,1.933898,110.483183
2,0,CT,Inpatient,8.752,3.877136,2.671706,0.689093
3,0,CT,Other/Unknown,8.752,0.218800,0.490007,2.239522
4,0,CT,Outpatient,8.752,0.901456,7.680908,8.520558


In [ ]:
pearson_chi2_b = ((nb_model.resid_pearson) ** 2).sum()
dispersion_b = pearson_chi2_b / nb_model.df_resid
print(f"Dispersion (Negative Binomial): {dispersion_b:.2f}")